In [11]:
from __future__ import print_function
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json
import os, sys

In [12]:
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import count, countDistinct
from pyspark.sql.functions import col
from pyspark.sql.functions import explode, expr
from pyspark.sql.functions import split
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
import pandas as pd
#from pyspark.sql import SQLContext
from pyspark.ml.feature import Word2Vec
from pyspark.mllib.linalg import Vectors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 10)
spark = SparkSession.builder.getOrCreate()

In [13]:
#Variables that contains the user credentials to access Twitter API 
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

#filename for training data
fileName = "trainingdata.json"
training = open(fileName,"a+") # we open file once with parameter "a+" that means apppend mode, when the file is exist instead of re-writing it, it will add new records to the end of file

#filename for testing data
fileNameT = "testingdata.json"
testing = open(fileNameT,"a+")

word2Vec = Word2Vec(vectorSize=5, minCount=0, inputCol="filtered", outputCol="features")

trainingTC = 0
numberTraining = 700
numberTesting = 300
testingTC = 0
isTraining = 1
isPrediction = 0
model =''
counter=0
#thedata=''

# Listen to Twitter

In [14]:
class StdOutListener(StreamListener):
    #event function when some data is streamed from Twitter
    def on_data(self,data):
        try:
            tweet = json.loads(data)
            text = tweet["text"]
            language_t =tweet["lang"]
            user_id = tweet["user"]["id"]
            date = tweet["user"]["created_at"]
            location= tweet["user"]["location"]
            
            #create tweet json object
            jsonObject = getJson(text, language_t, user_id, date, location)
            
            #saveToFile
            saveToFile(jsonObject)
            
        except:
            text = "NONE"
        return True 
    def on_error(self,status):
            print(status)

# Analyze Tweets

In [15]:

def trainData(cleanData): 
    global model
    global word2Vec
    
    model = word2Vec.fit(cleanData)
    result = model.transform(cleanData)
    result.coalesce(1).write.json("file:///Users/Laith/Downloads/liveTrainwordcloud.json")
    k = 20
    kmeans = KMeans().setK(k).setSeed(1)
    model = kmeans.fit(result.cache())
    cerror = model.computeCost(result.cache())
    print("Error:")
    print(cerror)
    print("-----")
    centers = model.clusterCenters()
    for center in centers:
        print(center)
    
def predictData(testData):
    global model
    global stream
    global word2Vec 
    print(testData.count())
    stream.disconnect()
    
   
    testmodel = word2Vec.fit(testData)
    result = testmodel.transform(testData)
    
    predictions = model.transform(result.cache())
    print(predictions.groupBy("prediction").count().sort(col("count").desc()).show())
    print("DONE!")
    
    #df = predictions.select("*").where("prediction == 0 OR prediction == 5 OR prediction == 12 OR prediction == 2")
    predictions.coalesce(1).write.json("file:///Users/Laith/Downloads/liveTestwordcloud.json")


    


    
    
   

# Clean and Prepare

In [16]:
def cleanData(jsonObject):
    global isPrediction
    #Language Cleaning
    jsonObject = jsonObject.where("tweet_lang=='en'")
    jsonObject = jsonObject.drop("tweet_lang")
    #Spammers Cleaning
    jsonObject = jsonObject.where("date NOT LIKE '%2019' OR date NOT LIKE '%Dec%'")
    jsonObject = jsonObject.where("date NOT LIKE '%2019' OR date NOT LIKE '%Nov%'")
    jsonObject = jsonObject.drop('date')
    
    #Location Cleaning
    jsonObject = jsonObject.select("*").where("location LIKE '%UK%' OR location LIKE '%England%' OR location LIKE '%Ireland%' OR location LIKE '%Scotland%' OR location LIKE '%Wales%'")
    #Text Cleaning
    regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
    regexTokenized = regexTokenizer.transform(jsonObject)
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    cleanData=remover.transform(regexTokenized) 
    if isPrediction == 1:
        print("Train Data")
        print("----------")
        trainData(cleanData)
    else:
        print("Predict Clusters!")
        print("----------")
        predictData(cleanData)
   

        
    

In [17]:
def prepareData():
    global isPrediction
    global thedata
    if isPrediction==1:
        tweetsDF = spark.read.json("file:///Users/Laith/Downloads/trainingdata.json")
        print("Cleaning Training Data")
        print("----------")
        cleanData(tweetsDF)
    else:
        tweetsDF = spark.read.json("file:///Users/Laith/Downloads/testingdata.json")
        #print(thedata)
        #tweetsDF = spark.read.text(thedata)
        print("Cleaning Testing Data")
        print("----------")
        cleanData(tweetsDF)

# Collect Tweets

In [18]:
def getJson(text, language_t, user_id, date, location):
        
        jsonObject = {}
        #info on the tweet
        jsonObject["text"] = text
        jsonObject["tweet_lang"] = language_t
        jsonObject["userId"]=user_id
        jsonObject["date"] = date
        jsonObject["location"]=location
        return json.dumps(jsonObject,ensure_ascii=False)

In [19]:
def saveToFile(jsonObject):
    global trainging
    global testing
    global stream
    global trainingTC
    global numberTraining
    global numberTesting
    global testingTC
    global isTraining
    global isPrediction
   # global thedata
    
    
    if isTraining==1:
        if trainingTC<numberTraining:
            if trainingTC == 0:
                trainingData = jsonObject
            else:
                trainingData ="\n"+ str(jsonObject)
            training.write(trainingData)
            trainingTC = trainingTC + 1
        else:
            print("Done Collecting Training Data")
            print("----------")
            isTraining = 0
            isPrediction = 1
            training.close()
            prepareData()
    if isPrediction==1:
        if testingTC<numberTesting:
            if testingTC == 0:
                #thedata = str(jsonObject) 
                testingData = jsonObject
            else:
                #thedata +="\n"
                #thedata +=str(jsonObject) 
                testingData ="\n"+ str(jsonObject)
            testing.write(testingData)
            testingTC = testingTC + 1
        else:
            print("Done Collecting Testing Data")
            print("----------")
            isTraining = 0
            isPrediction = 0
            testing.close()
            prepareData()
  

# Streaming Tweets

In [20]:
#This handles Twitter authetification and the connection to Twitter Streaming API
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)

#This line filter Twitter Streams to capture data by the keywords: 'Brexit'
stream.filter(track=['Brexit'])

Done Collecting Training Data
----------
Cleaning Training Data
----------
Train Data
----------
Error:
0.5063632462220918
-----
[-0.00675108 -0.00963421 -0.02127034  0.01712254  0.04468667]
[ 0.35265676 -0.23838489 -0.00085204 -0.01983175  0.54192808]
[ 0.02390559 -0.04388993 -0.06147818 -0.02891521 -0.15553067]
[ 0.07084669 -0.08673587 -0.17668829 -0.13037618 -0.47756792]
[ 0.05102493 -0.10541393 -0.04352352  0.07681057  0.27517457]
[ 0.21635589 -0.17140996 -0.06887425  0.00842998  0.46643025]
[ 0.05442541 -0.07964405 -0.12153011  0.00188124  0.04876418]
[ 0.02669956 -0.04768196 -0.09399693 -0.06851498 -0.25620242]
[ 0.06389806 -0.07907865 -0.05199772  0.00561332  0.11501992]
[ 0.04222893 -0.04004376 -0.02959105  0.01462069  0.04852524]
[ 0.14538063 -0.10207865 -0.04279329  0.05143618  0.28413789]
[ 0.04943365 -0.06304328 -0.07063565 -0.01523985 -0.04247486]
[ 0.02259919 -0.04760888 -0.04058219  0.02770113  0.02861752]
[ 0.02586812 -0.05570877 -0.03858688  0.02776076  0.11628708]
[-0